In [1]:
library(tidyverse)
library(caret)
library(glmnet)
library(dplyr)
#library(psych) 
library(lme4)
library(e1071)
library(GGally)
options(warn=-1)
options(scipen=999)
#library(crosstable)

── Attaching packages ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.2.1     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

Warning message:
“package ‘tibble’ was built under R version 4.2.3”
── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘caret’ was built under R version 4.2.3”
Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked fro

In [3]:
seed_list<-c(1001,1111,1221,1331,1441,1551,1661,1771,1881,1991)
cut_off<-c(0,10,50,100)#,500,1000)
time_off<-c(10000,10,20)#,50,100)
#time_off<-c(10)

lambda <- 10^seq(-3, 3, length = 100)
alpha<-seq(0,1,length=10)

In [4]:
library(doParallel)
cl <- makePSOCKcluster(30)
registerDoParallel(cl)

Loading required package: foreach


Attaching package: ‘foreach’


The following objects are masked from ‘package:purrr’:

    accumulate, when


Loading required package: iterators

Loading required package: parallel



In [5]:
m<-1
list_of_vi=list()
corpus_list<-c("coha")
list_of_rsqr=list()
ratings_list<-c("reddy","cordeiro90","cordeiro100")
tagged_list<-c("UnTagged")

ppmi_setting_list<-c("PPMI")
comp_setting_list<-c("Aware","Agnostic")
feature_setting_list<-c("woSetting","withSetting")
caret_spearman <- function(data, lev = NULL, model = NULL) {
  spearman_val <- cor(x = data$pred, y = data$obs, method = "spearman")
  c(Spearman = spearman_val)
}

In [6]:
for (c in corpus_list){
for (t in tagged_list) {
  for (f in feature_setting_list) {
  for (p in ppmi_setting_list){
    for (a in comp_setting_list){
      for (i in time_off) {
        
        for (j in cut_off) {
          

            print(paste0(c," ",p," ",a," ",f," ",t," ",i," ",j))
            
            na_df<-read.csv(paste0("/data/dharp/compounds/datasets/",c,"/features_Compound",a,"_",f,"_",p,"_",t,"_",i,"_",j,"_na.csv"),sep = '\t')
            nona_df<-read.csv(paste0("/data/dharp/compounds/datasets/",c,"/features_Compound",a,"_",f,"_",p,"_",t,"_",i,"_",j,"_med.csv"),sep = '\t')
            
            for (r in ratings_list){
              na_df<-na_df %>% filter(source==r)
              nona_df<-nona_df %>% filter(source==r)
              
              trainX_na<-na_df %>% select(-c(compositionality,modifier,head,avgModifier,stdevModifier,avgHead,stdevHead,stdevHeadModifier,compound,is_adj,source)) %>% select(-one_of("comp_freq_bins"))
              trainY_na<-na_df %>% select(compositionality)
              trainY_na<-trainY_na$compositionality
              
              
              trainX_nona<-nona_df %>% select(-c(compositionality,modifier,head,avgModifier,stdevModifier,avgHead,stdevHead,stdevHeadModifier,compound,is_adj,source)) %>% select(-one_of("comp_freq_bins"))
              trainY_nona<-nona_df %>% select(compositionality)
              trainY_nona<-trainY_nona$compositionality
              
              if (dim(trainX_nona)[1]<10) {
                break
              }
              for (s in seed_list)  {
                set.seed(s)
                seeds <- vector(mode = "list", length = 14)
                for(z in 1:13) seeds[[z]] <- sample.int(n=1000, 10)
                #for the last model
                seeds[[14]]<-sample.int(1000, 1)
                
                ridge_nona <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Rsquared",
                                    trControl = trainControl("cv",number = 10, search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = 0, lambda = lambda),
                                    preProc = c("nzv","center", "scale")
                )
                
                elastic_nona <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Rsquared",
                                      trControl = trainControl("cv", number = 10,search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = alpha, lambda = lambda),
                                      preProc = c("nzv","center", "scale")
                )
                
                lasso_nona <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Rsquared",
                                    trControl = trainControl("cv", number = 10,search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = 1, lambda = lambda),
                                    preProc = c("nzv","center", "scale")
                )
                
                
                ridge_na <- train(trainX_na,trainY_na,method = "glmnet",metric = "Rsquared",
                                  trControl = trainControl("cv",number = 10, search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = 0, lambda = lambda),
                                  preProcess = c("nzv","medianImpute", "center", "scale")
                )
                
                elastic_na <- train(trainX_na,trainY_na,method = "glmnet",metric = "Rsquared",
                                    trControl = trainControl("cv", number = 10,search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = alpha, lambda = lambda),
                                    preProcess = c("nzv","medianImpute", "center", "scale")
                )
                
                lasso_na <- train(trainX_na,trainY_na,method = "glmnet",metric = "Rsquared",
                                  trControl = trainControl("cv", number = 10,search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = 1, lambda = lambda),
                                  preProcess = c("nzv","medianImpute", "center", "scale")
                )
                
                
                ridge_nona_spearman <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Spearman",
                                             trControl = trainControl("cv",number = 10, search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = 0, lambda = lambda),
                                             preProc = c("nzv","center", "scale")
                )
                
                elastic_nona_spearman <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Spearman",
                                               trControl = trainControl("cv", number = 10,search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = alpha, lambda = lambda),
                                               preProc = c("nzv","center", "scale")
                )
                
                lasso_nona_spearman <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Spearman",
                                             trControl = trainControl("cv", number = 10,search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = 1, lambda = lambda),
                                             preProc = c("nzv","center", "scale")
                )
                
                
                ridge_na_spearman <- train(trainX_na,trainY_na,method = "glmnet",metric = "Spearman",
                                           trControl = trainControl("cv",number = 10, search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = 0, lambda = lambda),
                                           preProcess = c("nzv","medianImpute", "center", "scale")
                )
                
                elastic_na_spearman <- train(trainX_na,trainY_na,method = "glmnet",metric = "Spearman",
                                             trControl = trainControl("cv", number = 10,search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = alpha, lambda = lambda),
                                             preProcess = c("nzv","medianImpute", "center", "scale")
                )
                
                lasso_na_spearman <- train(trainX_na,trainY_na,method = "glmnet",metric = "Spearman",
                                           trControl = trainControl("cv", number = 10,search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = 1, lambda = lambda),
                                           preProcess = c("nzv","medianImpute", "center", "scale")
                )
                

                perf_elastic_na<-data.frame(feature=f,impute="r",tag=t,setting=a,dataset=r,corpus=c,ml_algo="elastic",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(elastic_na)[,"method"],TrainRsquared=getTrainPerf(elastic_na)[,"TrainRsquared"],TrainSpearman=getTrainPerf(elastic_na_spearman)[,"TrainSpearman"])
                perf_ridge_na<-data.frame(feature=f,impute="r",tag=t,setting=a,dataset=r,corpus=c,ml_algo="rigde",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(ridge_na)[,"method"],TrainRsquared=getTrainPerf(ridge_na)[,"TrainRsquared"],TrainSpearman=getTrainPerf(ridge_na_spearman)[,"TrainSpearman"])
                perf_lasso_na<-data.frame(feature=f,impute="r",tag=t,setting=a,dataset=r,corpus=c,ml_algo="lasso",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(lasso_na)[,"method"],TrainRsquared=getTrainPerf(lasso_na)[,"TrainRsquared"],TrainSpearman=getTrainPerf(lasso_na_spearman)[,"TrainSpearman"])
                
                
                perf_elastic_nona<-data.frame(feature=f,impute="python",tag=t,setting=a,dataset=r,corpus=c,ml_algo="elastic",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(elastic_nona)[,"method"],TrainRsquared=getTrainPerf(elastic_nona)[,"TrainRsquared"],TrainSpearman=getTrainPerf(elastic_nona_spearman)[,"TrainSpearman"])
                perf_ridge_nona<-data.frame(feature=f,impute="python",tag=t,setting=a,dataset=r,corpus=c,ml_algo="rigde",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(ridge_nona)[,"method"],TrainRsquared=getTrainPerf(ridge_nona)[,"TrainRsquared"],TrainSpearman=getTrainPerf(ridge_nona_spearman)[,"TrainSpearman"])
                perf_lasso_nona<-data.frame(feature=f,impute="python",tag=t,setting=a,dataset=r,corpus=c,ml_algo="lasso",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(lasso_nona)[,"method"],TrainRsquared=getTrainPerf(lasso_nona)[,"TrainRsquared"],TrainSpearman=getTrainPerf(lasso_nona_spearman)[,"TrainSpearman"])
                
                
                list_of_rsqr[[m]]<-perf_elastic_na
                m<-m+1
                
                list_of_rsqr[[m]]<-perf_elastic_nona
                m<-m+1
                
                list_of_rsqr[[m]]<-perf_ridge_na
                m<-m+1
                
                list_of_rsqr[[m]]<-perf_ridge_nona
                m<-m+1
                
                list_of_rsqr[[m]]<-perf_lasso_na
                m<-m+1 
                
                list_of_rsqr[[m]]<-perf_lasso_nona
                m<-m+1 
                
            }
            

          
            

              
              #varimp_elastic<-data.frame(model="norm",impute="r",tag=t,setting=a,dataset=k,corpus="coha",ml_algo="elastic",seed=s,timespan=i,cutoff=j,t(varImp(elastic)$importance),ppmi=p)
              #varimp_ridge<-data.frame(model="norm",impute="r",tag=t,setting=a,dataset=k,corpus="coha",ml_algo="ridge",seed=s,timespan=i,cutoff=j,t(varImp(ridge)$importance),ppmi=p)
              #varimp_lasso<-data.frame(model="norm",impute="r",tag=t,setting=a,dataset=k,corpus="coha",ml_algo="lasso",seed=s,timespan=i,cutoff=j,t(varImp(lasso)$importance),ppmi=p)
              #varimp_rpart<-data.frame(tag=t,setting=a,dataset=k,corpus="coha",ml_algo="rpart",seed=s,timespan=i,cutoff=j,t(varImp(rpart)$importance))
              
              #varimp_elastic_nona<-data.frame(model="norm",impute="python",tag=t,setting=a,dataset=k,corpus="coha",ml_algo="elastic",seed=s,timespan=i,cutoff=j,t(varImp(elastic_nona)$importance),ppmi=p)
              #varimp_ridge_nona<-data.frame(model="norm",impute="python",tag=t,setting=a,dataset=k,corpus="coha",ml_algo="ridge",seed=s,timespan=i,cutoff=j,t(varImp(ridge_nona)$importance),ppmi=p)
              #varimp_lasso_nona<-data.frame(model="norm",impute="python",tag=t,setting=a,dataset=k,corpus="coha",ml_algo="lasso",seed=s,timespan=i,cutoff=j,t(varImp(lasso_nona)$importance),ppmi=p)    
              #varimp_rpart_nona<-data.frame(tag=t,setting=a,dataset=k,corpus="coha",ml_algo="rpart_nona",seed=s,timespan=i,cutoff=j,t(varImp(rpart_nona)$importance))
              

              
              #list_of_rsqr[[m]]<-perf_rpart
              #list_of_vi[[m]]<-varimp_rpart
              #m<-m+1 
              
              #list_of_rsqr[[m]]<-perf_rpart_nona
              #list_of_vi[[m]]<-varimp_rpart_nona
              #m<-m+1
              
              
              #list_of_rsqr[[m]]<-perf_elastic_google
              #list_of_vi[[m]]<-varimp_elastic_google
              #m<-m+1
              
              #list_of_rsqr[[m]]<-perf_ridge_google
              #list_of_vi[[m]]<-varimp_ridge_google
              #m<-m+1
              
              #list_of_rsqr[[m]]<-perf_lasso_google
              #list_of_vi[[m]]<-varimp_lasso_google
              #m<-m+1 
              
              #list_of_rsqr[[m]]<-perf_rpart_google
              #list_of_vi[[m]]<-varimp_rpart_google
              #m<-m+1 
              
              #list_of_rsqr[[m]]<-perf_rpartnona_google
              #list_of_vi[[m]]<-varimp_rpartnona_google
              #m<-m+1 
            }
          
        }
      }
    }
  }
  }
}
}

[1] "coha PPMI Aware woSetting UnTagged 10000 0"
[1] "coha PPMI Aware woSetting UnTagged 10000 10"
[1] "coha PPMI Aware woSetting UnTagged 10000 50"
[1] "coha PPMI Aware woSetting UnTagged 10000 100"
[1] "coha PPMI Aware woSetting UnTagged 10 0"
[1] "coha PPMI Aware woSetting UnTagged 10 10"
[1] "coha PPMI Aware woSetting UnTagged 10 50"
[1] "coha PPMI Aware woSetting UnTagged 10 100"
[1] "coha PPMI Aware woSetting UnTagged 20 0"
[1] "coha PPMI Aware woSetting UnTagged 20 10"
[1] "coha PPMI Aware woSetting UnTagged 20 50"
[1] "coha PPMI Aware woSetting UnTagged 20 100"
[1] "coha PPMI Agnostic woSetting UnTagged 10000 0"
[1] "coha PPMI Agnostic woSetting UnTagged 10000 10"
[1] "coha PPMI Agnostic woSetting UnTagged 10000 50"


In [10]:
google_df <-read.csv(paste0("/data/dharp/compounds/datasets/coha/features_CompoundAware_withSetting_PPMI_UnTagged_10_0_med.csv"),sep = '\t')

#google_df$compound_label<-cut_number(google_df$compositionality,n=3,labels=c("low","med","high"))
#google_df$corpus<-"google"

#combined_df<-rbind(coha_df,google_df)

#combined_df %>% filter(source=="reddy") %>%ggplot(aes(fill=compound_label,x=comp_tf.0)) + facet_grid(~corpus)+geom_density(alpha = 0.3) + theme_bw()

reddy_df<-google_df %>% filter(source=="reddy")

#cordeiro90_df<-google_df %>% filter(source=="cordeiro90")
#cordeiro100_df<-google_df %>% filter(source=="cordeiro100")

trainX<-reddy_df %>% select(-c(compositionality,modifier,head,avgModifier,stdevModifier,avgHead,stdevHead,stdevHeadModifier,compound,is_adj,source)) %>% select(-one_of("comp_freq_bins"))
trainY<-reddy_df %>% select(compositionality)
trainY<-trainY$compositionality


for (s in seed_list)  {
  set.seed(s)
  seeds <- vector(mode = "list", length = 14)
  for(z in 1:13) seeds[[z]] <- sample.int(n=1000, 10)
  #for the last model
  seeds[[14]]<-sample.int(1000, 1)

  ridge_nona_coha <- train(trainX,trainY,method = "glmnet",metric = "Rsquared",
                           trControl = trainControl("cv",number = 10, search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = 0, lambda = lambda),
                           preProc = c("nzv","center", "scale")
  )
  print(getTrainPerf(ridge_nona_coha)[,"TrainRsquared"])

}

[1] 0.5147677
[1] 0.4472353
[1] 0.4870363
[1] 0.5061195
[1] 0.5017567
[1] 0.4814031
[1] 0.4848378
[1] 0.531367
[1] 0.491554
[1] 0.4797273


In [27]:
varImp_df$names <- rownames(varImp_df)

In [28]:
varImp_df[order(-varImp_df$Overall), ]

,Overall,names
,<dbl>,<chr>
comp_tf.2010,100.00000,comp_tf.2010
log_comp_freq.2010,99.98601,log_comp_freq.2010
comp_tf.2000,96.91917,comp_tf.2000
log_comp_freq.2000,96.89357,log_comp_freq.2000
comp_freq.2010,92.27417,comp_freq.2010
comp_freq.2000,86.45822,comp_freq.2000
log_comp_freq.1980,83.94510,log_comp_freq.1980
comp_tf.1980,83.86297,comp_tf.1980
local_mi.2010,81.81439,local_mi.2010


In [153]:
varimp_df<-bind_rows(list_of_vi)
varimp_df$cutoff<-as.factor(varimp_df$cutoff)
varimp_df[is.na(varimp_df)] <- 0

In [7]:
rsquared_df<-bind_rows(list_of_rsqr)
rsquared_df$cutoff<-as.factor(rsquared_df$cutoff)

In [8]:
rsquared_df[order(-rsquared_df$TrainSpearman), ]

,feature,impute,tag,setting,dataset,corpus,ml_algo,n,seed,timespan,cutoff,ppmi,method,TrainRsquared,TrainSpearman
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<fct>,<chr>,<chr>,<dbl>,<dbl>
277,woSetting,r,UnTagged,Aware,reddy,coha,elastic,91,1661,10,0,PPMI,glmnet,0.6548240,0.9000000
279,woSetting,r,UnTagged,Aware,reddy,coha,rigde,91,1661,10,0,PPMI,glmnet,0.6548240,0.9000000
271,woSetting,r,UnTagged,Aware,reddy,coha,elastic,91,1551,10,0,PPMI,glmnet,0.7280827,0.8809524
273,woSetting,r,UnTagged,Aware,reddy,coha,rigde,91,1551,10,0,PPMI,glmnet,0.7280827,0.8809524
488,woSetting,python,UnTagged,Aware,reddy,coha,elastic,91,1111,20,0,PPMI,glmnet,0.4619143,0.7782495
517,woSetting,r,UnTagged,Aware,reddy,coha,elastic,91,1661,20,0,PPMI,glmnet,0.5659343,0.7500000
519,woSetting,r,UnTagged,Aware,reddy,coha,rigde,91,1661,20,0,PPMI,glmnet,0.5659343,0.7500000
584,woSetting,python,UnTagged,Aware,reddy,coha,elastic,91,1771,20,10,PPMI,glmnet,0.5446829,0.7326190
499,woSetting,r,UnTagged,Aware,reddy,coha,elastic,91,1331,20,0,PPMI,glmnet,0.4911294,0.7154083


In [9]:
rsquared_df[order(-rsquared_df$TrainRsquared), ]

,feature,impute,tag,setting,dataset,corpus,ml_algo,n,seed,timespan,cutoff,ppmi,method,TrainRsquared,TrainSpearman
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<fct>,<chr>,<chr>,<dbl>,<dbl>
271,woSetting,r,UnTagged,Aware,reddy,coha,elastic,91,1551,10,0,PPMI,glmnet,0.7280827,0.8809524
273,woSetting,r,UnTagged,Aware,reddy,coha,rigde,91,1551,10,0,PPMI,glmnet,0.7280827,0.8809524
277,woSetting,r,UnTagged,Aware,reddy,coha,elastic,91,1661,10,0,PPMI,glmnet,0.6548240,0.9000000
279,woSetting,r,UnTagged,Aware,reddy,coha,rigde,91,1661,10,0,PPMI,glmnet,0.6548240,0.9000000
511,woSetting,r,UnTagged,Aware,reddy,coha,elastic,91,1551,20,0,PPMI,glmnet,0.6135620,0.7142857
513,woSetting,r,UnTagged,Aware,reddy,coha,rigde,91,1551,20,0,PPMI,glmnet,0.6135620,0.7142857
517,woSetting,r,UnTagged,Aware,reddy,coha,elastic,91,1661,20,0,PPMI,glmnet,0.5659343,0.7500000
519,woSetting,r,UnTagged,Aware,reddy,coha,rigde,91,1661,20,0,PPMI,glmnet,0.5659343,0.7500000
337,woSetting,r,UnTagged,Aware,reddy,coha,elastic,91,1661,10,10,PPMI,glmnet,0.5629563,0.6014094


In [155]:
write.csv(rsquared_df,"rsquared_Tagged_PPMI_Aware.csv",row.names = FALSE)

In [156]:
write.csv(varimp_df,"varimp_Tagged_PPMI_Aware.csv",row.names = FALSE)